In [1]:
import numpy as np
import pandas as pd 
from math import isnan
import jieba 
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from collections import Counter
txt = pd.read_excel('数据比较好的.xlsx')
#X = txt['商品描述']
Y = txt['标签']

c:\python3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
frame = pd.DataFrame(columns=['商品描述','标签'])
dic = Counter(Y)
for i in dic.keys():
    data = txt.loc[txt['标签'] == i]
    if dic[i] > 200:
        frame = pd.concat([frame,data[:200]],axis = 0)
    else:
        frame = pd.concat([frame,data],axis = 0)
frame = frame.reset_index(drop=True)

In [13]:
vectorizer = CountVectorizer()
fre_matrix = vectorizer.fit_transform(frame['商品描述'])
feature_names  = vectorizer.get_feature_names()
word_index_dic = vectorizer.vocabulary_

In [5]:
length = (fre_matrix.shape)[1]
newd = {v:k for k,v in word_index_dic.items()}
bag_words = []

In [6]:
n = 0
for label in Counter(frame['标签']).keys():
    if n % 100 == 0:
        print(n)
    choose = frame.loc[frame['标签'] == label].index
    A = fre_matrix[choose].sum(axis=0)
    C = np.zeros((1,length),np.int64)
    index_list = [] 
    for i in range(length):
        if A[0,i] != 0:
            index_list.append(i)
            C[0,i] = len(choose) - A[0,i]
    choose_1 = frame.loc[frame['标签'] != label].index
    B_ = np.zeros((1,length),np.int64)
    for index in choose_1:
        B_ = fre_matrix[index] + B_
    B = np.zeros((1,length),np.int64)
    for index in index_list:
        B[0,index] = B_[0,index]
    D = np.zeros((1,length),np.int64)
    for i in range(length):
        if B[0,i] != 0:
            D[0,i] = len(choose_1) - B[0,i]
    AD_BC_2 = (np.array(A)*np.array(D) - np.array(B)*np.array(C))**2
    chi2 =  AD_BC_2  / ((np.array(A) + np.array(B))*(np.array(C)+ np.array(D)))
    hope_index = {}
    for i in range(length):
        if isnan(chi2[0,i]) == False:
            hope_index[i] = chi2[0,i]
    li = sorted(hope_index.items(),key = lambda x:x[1],reverse = True)
    bag_words.append(newd[li[0][0]])
    n = n + 1

0


c:\python3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in true_divide


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200


In [7]:
good_words = list(set(bag_words))
print(len(bag_words),len(set(bag_words)))

2262 2144


In [8]:
print(len(frame),len(Counter(frame['标签']).keys()))

61467 2262


In [16]:
for row in range((fre_matrix.shape)[0]):
    for word in frame['商品描述'][row].split():
        if word in good_words:
            col = word_index_dic[word]
            fre_matrix[row,col] += 0.5

In [17]:
X_train,X_test,Y_train,Y_test = train_test_split(fre_matrix,frame['标签'],test_size = 0.2)
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, Y_train)# coding:utf-8
y_pred = classifier.predict(X_test)
print(np.mean(y_pred == Y_test))

0.8261753700992354


In [20]:
from xgboost import XGBClassifier

In [21]:
classifier = XGBClassifier()
classifier.fit(X_train, Y_train)# coding:utf-8
y_pred = classifier.predict(X_test)
print(np.mean(y_pred == Y_test))

0.648934439564015


c:\python3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [ ]:
这个结果是加一的

In [13]:
X_train,X_test,Y_train,Y_test = train_test_split(fre_matrix,frame['标签'],test_size = 0.2)
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, Y_train)# coding:utf-8
y_pred = clxassifier.predict(X_test)
print(np.mean(y_pred == Y_test))

0.8257292445774121


# 超过一百的取前一百条，label再分词结果里面的。词频 + 开方

In [11]:
len(frame['标签'])

53477

In [35]:
def chi2(label):
    choose = frame.loc[frame['标签'] == label].index
    A = fre_matrix[choose].sum(axis=0)
    C = np.zeros((1,length),np.int64)
    index_list = [] 
    for i in range(length):
        if A[0,i] != 0:
            index_list.append(i)
            C[0,i] = len(choose) - A[0,i]
    choose_1 = frame.loc[frame['标签'] != label].index
    B_ = np.zeros((1,length),np.int64)
    for index in choose_1:
        B_ = fre_matrix[index] + B_
    B = np.zeros((1,length),np.int64)
    for index in index_list:
        B[0,index] = B_[0,index]
    D = np.zeros((1,length),np.int64)
    for i in range(length):
        if B[0,i] != 0:
            D[0,i] = len(choose_1) - B[0,i]
    AD_BC_2 = (np.array(A)*np.array(D) - np.array(B)*np.array(C))**2
    chi2 =  AD_BC_2  / ((np.array(A) + np.array(B))*(np.array(C)+ np.array(D)))
    hope_index = {}
    for i in range(length):
        if isnan(chi2[0,i]) == False:
            hope_index[i] = chi2[0,i]
    li = sorted(hope_index.items(),key = lambda x:x[1],reverse = True)
    for i in range(10):
        print(newd[li[i][0]])